### Imports

In [1]:
from pyod.models.ecod import ECOD
from mpl_toolkits import mplot3d
from sklearn.metrics import silhouette_score
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from utils.utils import *

import hdbscan
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import sys

%load_ext autotime

/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/li

time: 177 µs (started: 2023-06-12 21:59:49 -05:00)


### Parameters

In [2]:
import json
  
# Opening JSON file
file = open("parameters.json")
params = json.load(file)

exp = params["exp"]
print("Exp:\t\t", exp)

method = params["method"]
print("Method:\t\t", method)

dimension = params["dimension"]
print("Dimension:\t", dimension)

groups_id = params["groups_id"]
print("Groups id:\t", groups_id)

subgroups_id = params["subgroups_id"]
print("Subgroups id:\t", subgroups_id)

option = params["option"]
print("Option:\t\t", option)

if option:
    
    subgroups_id_op = {}
    for group in groups_id:
        subgroups_id_op[group] = [option]
else:
    subgroups_id_op = subgroups_id
print("Subgroups id op:", subgroups_id_op)



Exp:		 exp5
Method:		 vgae
Dimension:	 3
Groups id:	 ['s_WT', 's_zwf1^']
Subgroups id:	 {'s_WT': ['1', '2', '3', '4', '5'], 's_zwf1^': ['1', '2', '3']}
Option:		 dyn
Subgroups id op: {'s_WT': ['dyn'], 's_zwf1^': ['dyn']}
time: 2.42 ms (started: 2023-06-12 21:59:49 -05:00)


### Edge embeddings

In [3]:
# get edges embeddings

edge_embeddings_global(exp, method, groups_id, subgroups_id_op)

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [01:36<00:00, 48.38s/it]

time: 1min 36s (started: 2023-06-12 21:59:49 -05:00)


In [4]:
df_edge_embeddings = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_{}_{}_{}.csv".format(exp, method, groups_id[0], subgroups_id_op[groups_id[0]][0]), index_col=[0, 1])
df_edge_embeddings.head()

0         1         2
0 1  0.018940  1.906752  2.081278
  2  1.492338  0.203028  0.125830
  3  0.325967  0.585500  0.000082
  4  1.583131  1.956642  0.005975
  5  0.559704  0.003867  3.148295

time: 325 ms (started: 2023-06-12 22:01:26 -05:00)


### Concat edge embeddings

In [5]:
for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.DataFrame()
    k = 0
    for subgroup in tqdm(subgroups_id_op[group]):
        df_edge_embeddings = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_{}_{}_{}.csv".format(exp, method, group, subgroup), index_col=[0, 1])
        df_edge_embeddings["subgroup"] = [k] * len(df_edge_embeddings)
        df_edge_embeddings_concat = pd.concat([df_edge_embeddings_concat, df_edge_embeddings])
        k += 1
    
    df_edge_embeddings_concat.to_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index=True)

100%|██████████| 2/2 [00:06<00:00,  3.10s/it]

time: 6.21 s (started: 2023-06-12 22:01:26 -05:00)


In [6]:
df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, groups_id[0], option), index_col=[0, 1])
df_edge_embeddings_concat.head()

0         1         2  subgroup
0 1  0.018940  1.906752  2.081278         0
  2  1.492338  0.203028  0.125830         0
  3  0.325967  0.585500  0.000082         0
  4  1.583131  1.956642  0.005975         0
  5  0.559704  0.003867  3.148295         0

time: 309 ms (started: 2023-06-12 22:01:32 -05:00)


In [7]:
# plot edge embeddings concat

for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index_col=[0, 1])

    x = df_edge_embeddings_concat.iloc[:, 0]
    y = df_edge_embeddings_concat.iloc[:, 1]
    z = df_edge_embeddings_concat.iloc[:, 2]

    # Creating figure
    fig = plt.figure(figsize=(10, 7))
    ax = plt.axes(projection="3d")

    # Creating plot
    ax.scatter3D(x, y, z, c=df_edge_embeddings_concat.iloc[:, -1], alpha=0.1)
    # plt.title("Dimension: {}".format(dimension))

    # show plot
    plt.savefig("output/{}/plots/edge-embeddings_concat_{}_{}_{}.png".format(exp, method, group, option))
    # plt.show()
    plt.clf()

100%|██████████| 2/2 [00:30<00:00, 15.37s/it]


<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

time: 30.7 s (started: 2023-06-12 22:01:33 -05:00)


### Outliers detection

In [8]:
# Outlier detection (HDBSCAN)

""" df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index_col=[0, 1])

X_train = df_edge_embeddings_concat.iloc[:, :-1]
clusterer = hdbscan.HDBSCAN(min_cluster_size=100, core_dist_n_jobs=-1).fit(X_train)

threshold = pd.Series(clusterer.outlier_scores_).quantile(0.9)
outliers = np.where(clusterer.outlier_scores_ > threshold)[0]
print(len(outliers))
outliers

inliers = np.setdiff1d(np.arange(len(df_edge_embeddings_concat)), outliers)
print(len(inliers))
inliers """

' df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index_col=[0, 1])\n\nX_train = df_edge_embeddings_concat.iloc[:, :-1]\nclusterer = hdbscan.HDBSCAN(min_cluster_size=100, core_dist_n_jobs=-1).fit(X_train)\n\nthreshold = pd.Series(clusterer.outlier_scores_).quantile(0.9)\noutliers = np.where(clusterer.outlier_scores_ > threshold)[0]\nprint(len(outliers))\noutliers\n\ninliers = np.setdiff1d(np.arange(len(df_edge_embeddings_concat)), outliers)\nprint(len(inliers))\ninliers '

time: 3.7 ms (started: 2023-06-12 22:02:04 -05:00)


In [9]:
# outlier detection (ECOD)

dict_df_edge_embeddings_concat_outlier = {}
dict_df_edge_embeddings_concat_filter = {}

for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index_col=[0, 1])

    X_train = df_edge_embeddings_concat.iloc[:, :-1]

    clf = ECOD()
    clf.fit(X_train)

    X_train["labels"] = clf.labels_ # binary labels (0: inliers, 1: outliers)

    df_edge_embeddings_concat_filter = df_edge_embeddings_concat.copy()
    df_edge_embeddings_concat_filter["labels"] = clf.labels_
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["labels"] == 0]
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, :-1]

    dict_df_edge_embeddings_concat_outlier[group] = X_train
    dict_df_edge_embeddings_concat_filter[group] = df_edge_embeddings_concat_filter

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:01<00:00,  1.16it/s]

time: 1.74 s (started: 2023-06-12 22:02:04 -05:00)


In [10]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0         1         2  subgroup
0 1  0.018940  1.906752  2.081278         0
  2  1.492338  0.203028  0.125830         0
  5  0.559704  0.003867  3.148295         0
  6  0.033487  1.334406  0.371853         0
  7  0.032720  0.747871  1.385118         0

time: 7.65 ms (started: 2023-06-12 22:02:05 -05:00)


In [11]:
# plot outliers/inliers

for group in tqdm(groups_id):
    fig = plt.figure(figsize = (10, 7))
    ax = plt.axes(projection ="3d")

    df_aux = dict_df_edge_embeddings_concat_outlier[group]
    print("Total:", len(df_aux))
    
    temp = df_aux[df_aux["labels"] == 0]
    x = temp.iloc[:, 0]
    y = temp.iloc[:, 1]
    z = temp.iloc[:, 2]
    ax.scatter3D(x, y, z, c="red", alpha=0.005)
    print("Num. of inliers:", len(temp))

    temp = df_aux[df_aux["labels"] == 1]
    x = temp.iloc[:, 0]
    y = temp.iloc[:, 1]
    z = temp.iloc[:, 2]
    ax.scatter3D(x, y, z, c="gray", alpha=0.005)
    print("Num. of inliers:", len(temp))

    # show plot
    plt.savefig("output/{}/plots/edge-embeddings_outlier_{}_{}_{}.png".format(exp, method, group, option))
    # plt.show()
    plt.clf()

  0%|          | 0/2 [00:00<?, ?it/s]

Total: 752561
Num. of inliers: 677305
Num. of inliers: 75256


 50%|█████     | 1/2 [00:18<00:18, 18.55s/it]

Total: 450751
Num. of inliers: 405676
Num. of inliers: 45075


100%|██████████| 2/2 [00:29<00:00, 14.83s/it]


<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

time: 29.7 s (started: 2023-06-12 22:02:05 -05:00)


###  Filter common edges

In [12]:
print(subgroups_id)
print(subgroups_id_op)

{'s_WT': ['1', '2', '3', '4', '5'], 's_zwf1^': ['1', '2', '3']}
{'s_WT': ['dyn'], 's_zwf1^': ['dyn']}
time: 282 µs (started: 2023-06-12 22:02:35 -05:00)


In [13]:
# mapping idx with id

for group in tqdm(groups_id):
    dict_df_nodes = {}
    for subgroup in subgroups_id_op[group]:
        df_nodes = pd.read_csv("output/{}/preprocessing/graphs_data/nodes_data_{}_{}.csv".format(exp, group, subgroup),
                               dtype={"id": "string"})
        dict_df_nodes[subgroup] = df_nodes
    
    # mapping
    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
    list_index = []
    
    for row in tqdm(df_edge_embeddings_concat_filter.itertuples()):
        df_nodes = dict_df_nodes[subgroups_id_op[group][row[-1]]]
        list_index.append((df_nodes.iloc[row[0][0], -1], df_nodes.iloc[row[0][1], -1]))
    
    # set new index
    df_edge_embeddings_concat_filter.set_index([pd.Index(list_index)], inplace=True)

  0%|          | 0/2 [00:00<?, ?it/s]

677305it [00:18, 35695.81it/s]
405676it [00:12, 32264.48it/s]:19, 19.90s/it]
100%|██████████| 2/2 [00:33<00:00, 16.55s/it]

time: 33.1 s (started: 2023-06-12 22:02:35 -05:00)


In [14]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0         1         2  subgroup
A0 A2   0.018940  1.906752  2.081278         0
   A3   1.492338  0.203028  0.125830         0
   A10  0.559704  0.003867  3.148295         0
   A11  0.033487  1.334406  0.371853         0
   A13  0.032720  0.747871  1.385118         0

time: 5.27 ms (started: 2023-06-12 22:03:08 -05:00)


In [15]:
# format id
if option:
    for group in tqdm(groups_id):
        # format
        df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
        list_index = []

        for row in tqdm(df_edge_embeddings_concat_filter.itertuples()):
            list_index.append((row[0][0][1:], row[0][1][1:]))
        
        # set new index
        df_edge_embeddings_concat_filter.set_index([pd.Index(list_index)], inplace=True)
        df_edge_embeddings_concat_filter

  0%|          | 0/2 [00:00<?, ?it/s]

677305it [00:00, 876622.73it/s]
405676it [00:00, 1134943.24it/s]1,  1.76s/it]
100%|██████████| 2/2 [00:02<00:00,  1.43s/it]

time: 2.88 s (started: 2023-06-12 22:03:08 -05:00)


In [16]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0         1         2  subgroup
0 2   0.018940  1.906752  2.081278         0
  3   1.492338  0.203028  0.125830         0
  10  0.559704  0.003867  3.148295         0
  11  0.033487  1.334406  0.371853         0
  13  0.032720  0.747871  1.385118         0

time: 6.23 ms (started: 2023-06-12 22:03:11 -05:00)


In [17]:
# filter diferente edges

if option:
    for group in tqdm(groups_id):
        df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
        df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter.index.get_level_values(0) != df_edge_embeddings_concat_filter.index.get_level_values(1)]
        dict_df_edge_embeddings_concat_filter[group] = df_edge_embeddings_concat_filter

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00, 17.76it/s]

time: 129 ms (started: 2023-06-12 22:03:11 -05:00)


In [18]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
print(df_edge_embeddings_concat_filter.shape)
df_edge_embeddings_concat_filter.head()

(673736, 4)


0         1         2  subgroup
0 2   0.018940  1.906752  2.081278         0
  3   1.492338  0.203028  0.125830         0
  10  0.559704  0.003867  3.148295         0
  11  0.033487  1.334406  0.371853         0
  13  0.032720  0.747871  1.385118         0

time: 12.6 ms (started: 2023-06-12 22:03:11 -05:00)


In [19]:
# count edges and filter by count

dict_df_edges_filter = {}
for group in tqdm(groups_id):
    # read
    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]

    # format
    df_edge_embeddings_concat_filter.reset_index(inplace=True)
    df_edge_embeddings_concat_filter.rename(columns={"level_0": "source", "level_1": "target"}, inplace=True)
    df_edge_embeddings_concat_filter[["source", "target"]] = df_edge_embeddings_concat_filter[["source", "target"]].astype("string")
    
    # sort edges
    sort_df_edges(df_edge_embeddings_concat_filter)

    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[["source", "target"]].value_counts().to_frame()
    df_edge_embeddings_concat_filter.reset_index(inplace=True)
    df_edge_embeddings_concat_filter.columns = ["source", "target", "count"]
    
    # filter
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["count"] == len(subgroups_id[group])]
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, [0, 1]]
    dict_df_edges_filter[group] = df_edge_embeddings_concat_filter

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:01<00:00,  1.21it/s]

time: 1.67 s (started: 2023-06-12 22:03:11 -05:00)


In [20]:
df_edges_filter = dict_df_edges_filter[groups_id[0]]
df_edges_filter.head()

,source,target
0,25,293
1,457,547
2,22,99
3,542,658
4,316,460


time: 4 ms (started: 2023-06-12 22:03:13 -05:00)


In [21]:
df_edges_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 750 entries, 0 to 749
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  750 non-null    object
 1   target  750 non-null    object
dtypes: object(2)
memory usage: 17.6+ KB
time: 18.2 ms (started: 2023-06-12 22:03:13 -05:00)


In [22]:
# change data type
for group in tqdm(groups_id):
    df_edges_filter = dict_df_edges_filter[group]
    df_edges_filter[["source", "target"]] = df_edges_filter[["source", "target"]].astype("string")

100%|██████████| 2/2 [00:00<00:00, 173.03it/s]

time: 17.7 ms (started: 2023-06-12 22:03:13 -05:00)


In [23]:
df_edges_filter = dict_df_edges_filter[groups_id[0]]
df_edges_filter.head()

,source,target
0,25,293
1,457,547
2,22,99
3,542,658
4,316,460


time: 7.75 ms (started: 2023-06-12 22:03:13 -05:00)


In [24]:
df_edges_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 750 entries, 0 to 749
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  750 non-null    string
 1   target  750 non-null    string
dtypes: string(2)
memory usage: 17.6 KB
time: 12.1 ms (started: 2023-06-12 22:03:13 -05:00)


In [25]:
# get weight by subgroups

dict_df_edges_filter_weight = get_weight_global(dict_df_edges_filter, exp, groups_id, subgroups_id)
df_edges_filter_weight = dict_df_edges_filter_weight[groups_id[0]]
df_edges_filter_weight.head()

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:02<00:00,  1.07s/it]


,source,target,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5
424,100,564,0.985533,0.665725,0.573667,0.502396,0.987404
112,101,218,0.631039,0.904780,0.843647,0.763187,0.747235
18,101,261,0.586436,0.852571,0.542120,0.793204,0.533752
656,101,521,0.603449,0.541472,0.823422,0.771309,0.923551
476,101,875,0.658182,0.521673,0.975333,0.605163,0.602441


time: 2.17 s (started: 2023-06-12 22:03:13 -05:00)


In [26]:
df_edges_filter_weight = dict_df_edges_filter_weight[groups_id[0]]
df_edges_filter_weight.head()

,source,target,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5
424,100,564,0.985533,0.665725,0.573667,0.502396,0.987404
112,101,218,0.631039,0.904780,0.843647,0.763187,0.747235
18,101,261,0.586436,0.852571,0.542120,0.793204,0.533752
656,101,521,0.603449,0.541472,0.823422,0.771309,0.923551
476,101,875,0.658182,0.521673,0.975333,0.605163,0.602441


time: 7.15 ms (started: 2023-06-12 22:03:15 -05:00)


### Filter by STD and average weight

In [27]:
dict_df_common_edges = std_global(dict_df_edges_filter_weight, exp, method, groups_id, option, th=0.3, plot=True, save=True)
dict_df_common_edges[groups_id[0]].head()

 50%|█████     | 1/2 [00:00<00:00,  5.84it/s]

100%|██████████| 2/2 [00:00<00:00,  4.45it/s]


,source,target,weight
0,100,564,0.742945
1,101,218,0.777978
2,101,261,0.661617
3,101,521,0.732641
4,101,875,0.672559


<Figure size 640x480 with 0 Axes>

time: 467 ms (started: 2023-06-12 22:03:15 -05:00)


In [28]:
df_common_edges = pd.read_csv("output/{}/common_edges/common_edges_{}_{}_{}.csv".format(exp, method, groups_id[0], option),
                              dtype={"source": "string", "target": "string"})
df_common_edges.head()

,source,target,weight
0,100,564,0.742945
1,101,218,0.777978
2,101,261,0.661617
3,101,521,0.732641
4,101,875,0.672559


time: 13 ms (started: 2023-06-12 22:03:16 -05:00)


In [29]:
# show details

for group in tqdm(groups_id):
    df_common_edges = pd.read_csv("output/{}/common_edges/common_edges_{}_{}_{}.csv".format(exp, method, group, option))
    
    G = nx.from_pandas_edgelist(df_common_edges, "source", "target", edge_attr=["weight"])
    print("Group: {}".format(group))
    graph_detail(G)

100%|██████████| 2/2 [00:00<00:00, 68.35it/s]

Group: s_WT
Num. nodes: 747
Num. edges: 750

Group: s_zwf1^
Num. nodes: 997
Num. edges: 10037

time: 33.3 ms (started: 2023-06-12 22:03:16 -05:00)
